In [1]:
import numpy as np
#the output 

In [2]:
# path = "/home/xiao/Projects/predrnn-pytorch/data/moving-mnist-example/moving-mnist-test.npz"
# data = {}
# dat_1 = np.load(path)
# for key in dat_1.keys():
#     print ('key', key)
#     data[key] = dat_1[key]
# print ('data[clips]', data['clips'], data['clips'].shape)
# print ('data[dims]', data['dims'], data['dims'].shape)
# print ('data[input_raw_data]', data['input_raw_data'], data['input_raw_data'].shape, np.median(data['input_raw_data']), np.max(data['input_raw_data']))

# path = "/home/xiao/Projects/ST_LSTM_MP/Data/2D_map/Map_array/normalized_train_video.npz"
# data = np.load(path)
# for key in data.keys():
#     print ('key', key)

# print ('data[clips]', data['clips'].shape)
# print ('data[dims]', data['dims'])
# print ('data[input_raw_data]', data['input_raw_data'].shape, np.median(data['input_raw_data']), np.max(data['input_raw_data']))


In [3]:
import os.path as op
arr_dir = "../../Data/2D_map_64/Map_array"
total_maz = np.load(op.join(arr_dir, 'maz.npy'))
print ('total_maz', total_maz.shape)

total_maz (120, 64, 64)


In [4]:
train_path = np.load(op.join(arr_dir, "train_path.npy"))


print ('train_path', train_path.shape)


train_path (160000, 16, 2)


In [5]:
train_path = np.reshape(train_path, (100, -1, train_path.shape[1], train_path.shape[2]))
print ('train_path', train_path.shape, np.max(train_path), np.min(train_path))


#select the paths with length less than 12
all_map = []
for i in range(train_path.shape[0]):
    per_map = []
    for j in range(train_path.shape[1]):
        if train_path[i,j,-5,0] == train_path[i,j,-1,0] and train_path[i,j,-5,1] == train_path[i,j,-1,1]:
            per_map.append(train_path[i,j,:-4,:])
    print ('per_map', len(per_map))
    all_map.append(per_map[:1000])
all_map = np.array(all_map)
train_path = all_map
print ('new train_path', train_path.shape)

            

train_path (100, 1600, 16, 2) 63 0
per_map 1600
per_map 1599
per_map 1599
per_map 1599
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1599
per_map 1600
per_map 1599
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1599
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1599
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1599
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per_map 1599
per_map 1600
per_map 1600
per_map 1600
per_map 1600
per

In [6]:
# a = np.random.randint(0, 10, (3,3,3))
# print ('0', a)
# a = np.tile(a, (2,1, 1))
# print ('1', a, a.shape)

In [7]:
def augmentation(img, start_x, start_y, channel=1):
#     start_x, start_y = np.where(img[1]==255)
    h, w = 7, 7
#     for i range(-3, 4):
#         for j in range(-3, 4):
#             x, y = start_x + i, start_y + j
#             if img[0,x,y] != 255:
#                 img[1,x,y] = 255
#     start_x, start_y = np.where(img[2]==255)
    for i in range(-3, 4):
        for j in range(-3, 4):
            x, y = start_x + i, start_y + j
            if 0<=x<=(img.shape[-1]-1) and 0 <=y<=(img.shape[-1]-1) and img[0,x,y] != 255:
                img[channel,x,y] = 255
    return img
            
                

In [8]:
def generate_video(maz, path, seen=True, video_number=0):

    if not video_number:
        video_number = path.shape[1]
    for maz_id in range(path.shape[0]):
        video_per_maz = []
#         for task_id in range(path.shape[1]):
        for task_id in range(video_number):
            start, goal = path[maz_id, task_id, 0], path[maz_id, task_id, -1]
            maz_3channel = np.zeros((3, maz.shape[1], maz.shape[2]))
            if seen:
                maz_3channel[0] = np.copy(total_maz[maz_id])
            else:
                maz_3channel[0] = np.copy(total_maz[maz_id+100])
#             maz_3channel[1, start[0], start[1]] = 255
            maz_3channel[2, goal[0], goal[1]] = 255
            maz_3channel = augmentation(maz_3channel, goal[0], goal[1], 2)
            repeated_maz = np.tile(maz_3channel, (path.shape[2], 1, 1))
            
#             print ('repeated_maz', repeated_maz.shape)
            repeated_maz = np.reshape(repeated_maz, (path.shape[2], 3, maz.shape[1], maz.shape[2]))
#             print ('repeated_maz', repeated_maz.shape)
            for path_id in range(0, path.shape[2]):#-1):
                pos = path[maz_id, task_id, path_id]
                repeated_maz[path_id, 1, pos[0], pos[1]] = 255
                repeated_maz[path_id] = augmentation(repeated_maz[path_id], pos[0], pos[1], 1)
#                 repeated_maz = augmentation(repeated_maz, goal[0], goal[1])
                
#             repeated_maz[path_id+1, 1, pos[0], pos[1]] = 255
#             repeated_maz[path_id+1] = augmentation(repeated_maz[path_id+1], pos[0], pos[1], 1)
            
            
            video_per_maz.append(repeated_maz)
        video_per_maz = np.array(video_per_maz)
#         print ('video_per_maz', video_per_maz.shape)
        video_per_maz = np.expand_dims(video_per_maz, axis = 0)
        if maz_id == 0:
            videos = video_per_maz
        else:
            videos = np.concatenate((videos, video_per_maz), axis=0)
        
#         print ('video_per_maz', video_per_maz.shape)
        print ('videos', videos.shape)
#         videos.append(video_per_maz)
#     videos = np.stack(videos)
    
    return videos
    
                


In [9]:
num = 1000
train_videos = generate_video(total_maz, train_path, True, num)
video_num = train_videos.shape[0] * num
video_len = train_videos.shape[2]

videos (1, 1000, 12, 3, 64, 64)
videos (2, 1000, 12, 3, 64, 64)
videos (3, 1000, 12, 3, 64, 64)
videos (4, 1000, 12, 3, 64, 64)
videos (5, 1000, 12, 3, 64, 64)
videos (6, 1000, 12, 3, 64, 64)
videos (7, 1000, 12, 3, 64, 64)
videos (8, 1000, 12, 3, 64, 64)
videos (9, 1000, 12, 3, 64, 64)
videos (10, 1000, 12, 3, 64, 64)
videos (11, 1000, 12, 3, 64, 64)
videos (12, 1000, 12, 3, 64, 64)
videos (13, 1000, 12, 3, 64, 64)
videos (14, 1000, 12, 3, 64, 64)
videos (15, 1000, 12, 3, 64, 64)
videos (16, 1000, 12, 3, 64, 64)
videos (17, 1000, 12, 3, 64, 64)
videos (18, 1000, 12, 3, 64, 64)
videos (19, 1000, 12, 3, 64, 64)
videos (20, 1000, 12, 3, 64, 64)
videos (21, 1000, 12, 3, 64, 64)
videos (22, 1000, 12, 3, 64, 64)
videos (23, 1000, 12, 3, 64, 64)
videos (24, 1000, 12, 3, 64, 64)
videos (25, 1000, 12, 3, 64, 64)
videos (26, 1000, 12, 3, 64, 64)
videos (27, 1000, 12, 3, 64, 64)
videos (28, 1000, 12, 3, 64, 64)
videos (29, 1000, 12, 3, 64, 64)
videos (30, 1000, 12, 3, 64, 64)
videos (31, 1000, 1

In [10]:
train_videos = train_videos/255.0
np.save(op.join(arr_dir, "train_video.npy"), train_videos)
print ('train_video00', train_videos.shape)
print ('max min', train_videos.max(), train_videos.min())

train_video00 (100, 1000, 12, 3, 64, 64)
max min 1.0 0.0


In [11]:
train_videos = np.reshape(train_videos, (-1, 3, train_videos.shape[-2], train_videos.shape[-1])).astype(np.float32)
print ('train_video', train_videos.shape)

dims = np.array([3, train_videos.shape[-2], train_videos.shape[-1]])
dims = np.expand_dims(dims, axis=0)
print (dims)
input_seq = 5
input_start = np.arange(0, video_num*video_len, video_len).astype(np.int32)
output_start = np.arange(input_seq, video_num*video_len, video_len).astype(np.int32)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
# print (np.ones(video_num).astype(np.int32).shape)
input_start = np.stack((input_start, input_seq * np.ones(video_num).astype(np.int32)))
output_start = np.stack((output_start, (video_len-input_seq) * np.ones(video_num).astype(np.int32)))
input_start = np.transpose(input_start)
output_start = np.transpose(output_start)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
clips = np.stack((input_start, output_start))
print (clips.shape)
np.savez(op.join(arr_dir, "normalized_train_video.npz"), dims=dims, clips=clips, input_raw_data=train_videos)



train_video (1200000, 3, 64, 64)
[[ 3 64 64]]
(2, 100000, 2)


In [12]:
test_seen_file = op.join(arr_dir, "test_seen_path.npy")
test_seen_path = np.load(test_seen_file)
print ('test_seen_path', test_seen_path.shape)

test_seen_path = np.reshape(test_seen_path, (100, -1, test_seen_path.shape[1], test_seen_path.shape[2]))
print ('test_seen_path', test_seen_path.shape)

test_seen_path (40000, 13, 2)
test_seen_path (100, 400, 13, 2)


In [13]:
#######
all_map = []
for i in range(test_seen_path.shape[0]):
    per_map = []
    for j in range(test_seen_path.shape[1]):
        if test_seen_path[i,j,-2,0] == test_seen_path[i,j,-1,0] and test_seen_path[i,j,-2,1] == test_seen_path[i,j,-1,1]:
            per_map.append(test_seen_path[i,j,:-1,:])
    print ('per_map', len(per_map))
    all_map.append(per_map[:200])
all_map = np.array(all_map)
test_seen_path = all_map
print ('new test_seen_path', test_seen_path.shape)
#######

per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 398
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_map 400
per_

In [14]:
test_seen_videos = generate_video(total_maz, test_seen_path, True, 0)
video_num = test_seen_videos.shape[0] * test_seen_videos.shape[1]
video_len = test_seen_videos.shape[2]
test_seen_videos = test_seen_videos/255.0
np.save(op.join(arr_dir, "test_seen_video.npy"), test_seen_videos)

videos (1, 200, 12, 3, 64, 64)
videos (2, 200, 12, 3, 64, 64)
videos (3, 200, 12, 3, 64, 64)
videos (4, 200, 12, 3, 64, 64)
videos (5, 200, 12, 3, 64, 64)
videos (6, 200, 12, 3, 64, 64)
videos (7, 200, 12, 3, 64, 64)
videos (8, 200, 12, 3, 64, 64)
videos (9, 200, 12, 3, 64, 64)
videos (10, 200, 12, 3, 64, 64)
videos (11, 200, 12, 3, 64, 64)
videos (12, 200, 12, 3, 64, 64)
videos (13, 200, 12, 3, 64, 64)
videos (14, 200, 12, 3, 64, 64)
videos (15, 200, 12, 3, 64, 64)
videos (16, 200, 12, 3, 64, 64)
videos (17, 200, 12, 3, 64, 64)
videos (18, 200, 12, 3, 64, 64)
videos (19, 200, 12, 3, 64, 64)
videos (20, 200, 12, 3, 64, 64)
videos (21, 200, 12, 3, 64, 64)
videos (22, 200, 12, 3, 64, 64)
videos (23, 200, 12, 3, 64, 64)
videos (24, 200, 12, 3, 64, 64)
videos (25, 200, 12, 3, 64, 64)
videos (26, 200, 12, 3, 64, 64)
videos (27, 200, 12, 3, 64, 64)
videos (28, 200, 12, 3, 64, 64)
videos (29, 200, 12, 3, 64, 64)
videos (30, 200, 12, 3, 64, 64)
videos (31, 200, 12, 3, 64, 64)
videos (32, 200, 

In [15]:
test_seen_videos = np.reshape(test_seen_videos, (-1, 3, test_seen_videos.shape[-2], test_seen_videos.shape[-1])).astype(np.float32)

dims = np.array([3, test_seen_videos.shape[-2], test_seen_videos.shape[-1]])
dims = np.expand_dims(dims, axis=0)
# print (dims)
input_seq = 1
input_start = np.arange(0, video_num*video_len, video_len).astype(np.int32)
output_start = np.arange(input_seq, video_num*video_len, video_len).astype(np.int32)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
# print (np.ones(video_num).astype(np.int32).shape)
input_start = np.stack((input_start, input_seq * np.ones(video_num).astype(np.int32)))
output_start = np.stack((output_start, (video_len-input_seq) * np.ones(video_num).astype(np.int32)))
input_start = np.transpose(input_start)
output_start = np.transpose(output_start)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
clips = np.stack((input_start, output_start))
print (clips.shape)
np.savez(op.join(arr_dir, "normalized_test_seen_video.npz"), dims=dims, clips=clips, input_raw_data=test_seen_videos)


(2, 20000, 2)


In [16]:
test_unseen_file = op.join(arr_dir, "test_unseen_path.npy")
test_unseen_path = np.load(test_unseen_file)
print ('test_unseen_path', test_unseen_path.shape)

test_unseen_path = np.reshape(test_unseen_path, (20, -1, test_unseen_path.shape[1], test_unseen_path.shape[2]))
print ('test_unseen_path', test_unseen_path.shape)

test_unseen_path (8000, 14, 2)
test_unseen_path (20, 400, 14, 2)


In [17]:
test_unseen_videos = generate_video(total_maz, test_unseen_path, False, 0)
video_num = test_unseen_videos.shape[0] * test_unseen_videos.shape[1]
video_len = test_unseen_videos.shape[2]
test_unseen_videos = test_unseen_videos/255.0
np.save(op.join(arr_dir, "test_unseen_video.npy"), test_unseen_videos)

videos (1, 400, 14, 3, 64, 64)
videos (2, 400, 14, 3, 64, 64)
videos (3, 400, 14, 3, 64, 64)
videos (4, 400, 14, 3, 64, 64)
videos (5, 400, 14, 3, 64, 64)
videos (6, 400, 14, 3, 64, 64)
videos (7, 400, 14, 3, 64, 64)
videos (8, 400, 14, 3, 64, 64)
videos (9, 400, 14, 3, 64, 64)
videos (10, 400, 14, 3, 64, 64)
videos (11, 400, 14, 3, 64, 64)
videos (12, 400, 14, 3, 64, 64)
videos (13, 400, 14, 3, 64, 64)
videos (14, 400, 14, 3, 64, 64)
videos (15, 400, 14, 3, 64, 64)
videos (16, 400, 14, 3, 64, 64)
videos (17, 400, 14, 3, 64, 64)
videos (18, 400, 14, 3, 64, 64)
videos (19, 400, 14, 3, 64, 64)
videos (20, 400, 14, 3, 64, 64)


In [18]:
test_unseen_videos = np.reshape(test_unseen_videos, (-1, 3, test_unseen_videos.shape[-2], test_unseen_videos.shape[-1])).astype(np.float32)

dims = np.array([3, test_unseen_videos.shape[-2], test_unseen_videos.shape[-1]])
dims = np.expand_dims(dims, axis=0)
# print (dims)
input_seq = 1
input_start = np.arange(0, video_num*video_len, video_len).astype(np.int32)
output_start = np.arange(input_seq, video_num*video_len, video_len).astype(np.int32)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
# print (np.ones(video_num).astype(np.int32).shape)
input_start = np.stack((input_start, input_seq * np.ones(video_num).astype(np.int32)))
output_start = np.stack((output_start, (video_len-input_seq) * np.ones(video_num).astype(np.int32)))
input_start = np.transpose(input_start)
output_start = np.transpose(output_start)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
clips = np.stack((input_start, output_start))
print (clips.shape)
np.savez(op.join(arr_dir, "normalized_test_unseen_video.npz"), dims=dims, clips=clips, input_raw_data=test_unseen_videos)


(2, 8000, 2)
